<a href="https://colab.research.google.com/github/C23-PS396/LeftLovers-MachineLearning/blob/main/Content_Based_Filtering_Restaurant_Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# kaggle API
!pip install -q kaggle
from google.colab import files
files.upload()
!mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# download the data set and unzip
!kaggle datasets download -d ahmedshahriarsakib/uber-eats-usa-restaurants-menus
!unzip /content/uber-eats-usa-restaurants-menus.zip

In [ ]:
# libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

In [ ]:
# load the dataset and preview
df = pd.read_csv('/content/restaurants.csv')
df.head()

,id,position,name,score,ratings,category,price_range,full_address,zip_code,lat,lng
0,1,19,PJ Fresh (224 Daniel Payne Drive),NaN,NaN,"Burgers, American, Sandwiches",$,"224 Daniel Payne Drive, Birmingham, AL, 35207",35207,33.562365,-86.830703
1,2,9,J' ti`'z Smoothie-N-Coffee Bar,NaN,NaN,"Coffee and Tea, Breakfast and Brunch, Bubble Tea",NaN,"1521 Pinson Valley Parkway, Birmingham, AL, 35217",35217,33.583640,-86.773330
2,3,6,Philly Fresh Cheesesteaks (541-B Graymont Ave),NaN,NaN,"American, Cheesesteak, Sandwiches, Alcohol",$,"541-B Graymont Ave, Birmingham, AL, 35204",35204,33.509800,-86.854640
3,4,17,Papa Murphy's (1580 Montgomery Highway),NaN,NaN,Pizza,$,"1580 Montgomery Highway, Hoover, AL, 35226",35226,33.404439,-86.806614
4,5,162,Nelson Brothers Cafe (17th St N),4.7,22.0,"Breakfast and Brunch, Burgers, Sandwiches",NaN,"314 17th St N, Birmingham, AL, 35203",35203,33.514730,-86.811700


## EDA and Preprocessing

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40227 entries, 0 to 40226
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            40227 non-null  int64  
 1   position      40227 non-null  int64  
 2   name          40227 non-null  object 
 3   score         22254 non-null  float64
 4   ratings       22254 non-null  float64
 5   category      40204 non-null  object 
 6   price_range   33581 non-null  object 
 7   full_address  39949 non-null  object 
 8   zip_code      39940 non-null  object 
 9   lat           40227 non-null  float64
 10  lng           40227 non-null  float64
dtypes: float64(4), int64(2), object(5)
memory usage: 3.4+ MB


In [ ]:
# drop unecessary columns
df.drop(columns=['position', 'full_address', 'zip_code'], axis=1, inplace=True)

In [ ]:
# counting missing value percentage of each column
missing_df = (df.isnull().sum() / df.shape[0] * 100).reset_index().rename(columns={0: 'missing %'})
missing_df[missing_df['missing %'] > 0]

,index,missing %
2,score,44.678947
3,ratings,44.678947
4,category,0.057176
5,price_range,16.521242


In [ ]:
# change price range column type
df['price_range'].unique()

array(['$', nan, '$$', '$$$', '$$$$'], dtype=object)

In [ ]:
# dropping price range col
df.drop(columns=['price_range'], axis=1, inplace=True)

In [ ]:
# handling missing numerical values
for cat in ['score', 'ratings']:
  df[cat].fillna(df[cat].mean(), inplace=True)

In [ ]:
# handling missing categorical value

# inspect
df[df['category'].isnull()][['name']]

,name
978,Dunkin' (8171 Us Highway 431)
7106,Sam's Hot Dog Stand
11812,McMenamins Zeus Cafe
14400,Millwood Grocery
14527,Bulldog Liquor &amp; Wine
15051,A1 Hop Shop 144
15732,Liquor &amp; Liquor
21786,Yaki Sushi
24474,Sam's Kitchen
24483,Fabios NY Pizza


In [ ]:
# fill the null value with mode since replacing them one by one would be
# impractical
mode = df['category'].value_counts().index[0]
df['category'].fillna(mode, inplace=True)

In [ ]:
# recheck missing values
df.isnull().sum()

id          0
name        0
score       0
ratings     0
category    0
lat         0
lng         0
dtype: int64

### Picking the top 20 categories

In [ ]:
# get category unique values
cats_dup = df['category'].tolist()
cats_list = []
for line in cats_dup:
  for cat in line.split(', '):
    cats_list.append(cat)

# picking top 20 categories
cats_20 = pd.value_counts(cats_list)[:20].index
print("top 20 categories: \n" + ", ".join(cats_20))

top 20 categories: 
American, Burgers, Fast Food, Sandwich, Sandwiches, Healthy, Asian, Comfort Food, Family Meals, Mexican, Breakfast and Brunch, Desserts, Pizza, Salads, Chicken, Convenience, Italian, Everyday Essentials, Wings, Family Friendly


In [ ]:
# filtering out restaurants
ids = []
for cat in cats_20:
  ids += list(df[df['category'].str.contains(cat)].index)

print(f"The num of list restaurants {len(ids)}") 
unique_ids = set(ids)
print(f"The num of unique restaurants {len(unique_ids)}") 

The num of list restaurants 97651
The num of unique restaurants 37540


In [ ]:
# new dataframe
df = df.iloc[list(unique_ids)]
df.head()

,id,name,score,ratings,category,lat,lng
0,1,PJ Fresh (224 Daniel Payne Drive),4.560996,74.870989,"Burgers, American, Sandwiches",33.562365,-86.830703
1,2,J' ti`'z Smoothie-N-Coffee Bar,4.560996,74.870989,"Coffee and Tea, Breakfast and Brunch, Bubble Tea",33.583640,-86.773330
2,3,Philly Fresh Cheesesteaks (541-B Graymont Ave),4.560996,74.870989,"American, Cheesesteak, Sandwiches, Alcohol",33.509800,-86.854640
3,4,Papa Murphy's (1580 Montgomery Highway),4.560996,74.870989,Pizza,33.404439,-86.806614
4,5,Nelson Brothers Cafe (17th St N),4.700000,22.000000,"Breakfast and Brunch, Burgers, Sandwiches",33.514730,-86.811700


### One-hot encode the features

In [ ]:
# one hot encode
for cat in cats_20:
  df[cat] = df['category'].apply(lambda x: 1 if cat in x else 0)

df.head()

<ipython-input-96-3dc5e9ee8ce7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[cat] = df['category'].apply(lambda x: 1 if cat in x else 0)
<ipython-input-96-3dc5e9ee8ce7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[cat] = df['category'].apply(lambda x: 1 if cat in x else 0)


,id,name,score,ratings,category,lat,lng,American,Burgers,Fast Food,...,Breakfast and Brunch,Desserts,Pizza,Salads,Chicken,Convenience,Italian,Everyday Essentials,Wings,Family Friendly
0,1,PJ Fresh (224 Daniel Payne Drive),4.560996,74.870989,"Burgers, American, Sandwiches",33.562365,-86.830703,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,J' ti`'z Smoothie-N-Coffee Bar,4.560996,74.870989,"Coffee and Tea, Breakfast and Brunch, Bubble Tea",33.583640,-86.773330,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,3,Philly Fresh Cheesesteaks (541-B Graymont Ave),4.560996,74.870989,"American, Cheesesteak, Sandwiches, Alcohol",33.509800,-86.854640,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,Papa Murphy's (1580 Montgomery Highway),4.560996,74.870989,Pizza,33.404439,-86.806614,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,5,Nelson Brothers Cafe (17th St N),4.700000,22.000000,"Breakfast and Brunch, Burgers, Sandwiches",33.514730,-86.811700,0,1,0,...,1,0,0,0,0,0,0,0,0,0


## Merge with menu dataset

In [ ]:
menu_df = pd.read_csv('/content/restaurant-menus.csv')
menu_df.head()

,restaurant_id,category,name,description,price
0,1,Extra Large Pizza,Extra Large Meat Lovers,Whole pie.,15.99 USD
1,1,Extra Large Pizza,Extra Large Supreme,Whole pie.,15.99 USD
2,1,Extra Large Pizza,Extra Large Pepperoni,Whole pie.,14.99 USD
3,1,Extra Large Pizza,Extra Large BBQ Chicken &amp; Bacon,Whole Pie,15.99 USD
4,1,Extra Large Pizza,Extra Large 5 Cheese,Whole pie.,14.99 USD


In [ ]:
menu_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3375211 entries, 0 to 3375210
Data columns (total 5 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   restaurant_id  int64 
 1   category       object
 2   name           object
 3   description    object
 4   price          object
dtypes: int64(1), object(4)
memory usage: 128.8+ MB


In [ ]:
# preprocess
menu_df['price'] = menu_df['price'].str.replace(" USD", "")
menu_df['price'] = menu_df['price'].astype(float)
menu_df.dtypes

restaurant_id      int64
category          object
name              object
description       object
price            float64
dtype: object

In [ ]:
# get the price range
avg_menu = menu_df.groupby('restaurant_id')[['price']].mean().reset_index().rename(columns={'restaurant_id':'id', 'price':'avg_price'})
avg_menu.head()

,id,avg_price
0,1,5.663684
1,2,5.505333
2,3,10.762143
3,4,10.531892
4,5,4.532576


In [ ]:
# merge
df = pd.merge(df, avg_menu, how='left', on='id')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37540 entries, 0 to 37539
Data columns (total 28 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    37540 non-null  int64  
 1   name                  37540 non-null  object 
 2   score                 37540 non-null  float64
 3   ratings               37540 non-null  float64
 4   category              37540 non-null  object 
 5   lat                   37540 non-null  float64
 6   lng                   37540 non-null  float64
 7   American              37540 non-null  int64  
 8   Burgers               37540 non-null  int64  
 9   Fast Food             37540 non-null  int64  
 10  Sandwich              37540 non-null  int64  
 11  Sandwiches            37540 non-null  int64  
 12  Healthy               37540 non-null  int64  
 13  Asian                 37540 non-null  int64  
 14  Comfort Food          37540 non-null  int64  
 15  Family Meals       

In [ ]:
# filling the nan value for the missing values, fill them with mean
df['avg_price'].fillna(df['avg_price'].mean(), inplace=True)

In [ ]:
df['avg_price'].isnull().sum()

0

In [ ]:
df.head()

,id,name,score,ratings,category,lat,lng,American,Burgers,Fast Food,...,Desserts,Pizza,Salads,Chicken,Convenience,Italian,Everyday Essentials,Wings,Family Friendly,avg_price
0,1,PJ Fresh (224 Daniel Payne Drive),4.560996,74.870989,"Burgers, American, Sandwiches",33.562365,-86.830703,1,1,0,...,0,0,0,0,0,0,0,0,0,5.663684
1,2,J' ti`'z Smoothie-N-Coffee Bar,4.560996,74.870989,"Coffee and Tea, Breakfast and Brunch, Bubble Tea",33.583640,-86.773330,0,0,0,...,0,0,0,0,0,0,0,0,0,5.505333
2,3,Philly Fresh Cheesesteaks (541-B Graymont Ave),4.560996,74.870989,"American, Cheesesteak, Sandwiches, Alcohol",33.509800,-86.854640,1,0,0,...,0,0,0,0,0,0,0,0,0,10.762143
3,4,Papa Murphy's (1580 Montgomery Highway),4.560996,74.870989,Pizza,33.404439,-86.806614,0,0,0,...,0,1,0,0,0,0,0,0,0,10.531892
4,5,Nelson Brothers Cafe (17th St N),4.700000,22.000000,"Breakfast and Brunch, Burgers, Sandwiches",33.514730,-86.811700,0,1,0,...,0,0,0,0,0,0,0,0,0,4.532576


## Creating User transaction Dataframe



The features of the users will be:

Not used for training
1.   id (restaurant_id)

Used for training
1.   Age ranging from 18 to 60
2.   Per Transaction spendings 15 to 100 usd avg
3.   The number of transactions done for each food category ranging from 1 - 100

for now there'll be 300 users who have rated 20 to 300 restaurants



In [ ]:
# creating user data frame
users_data = np.array([]).reshape(-1,23)
for i in range(300):
  num_res = np.random.randint(20, 300)
  res_ids = np.random.randint(0, df.shape[0], size=(num_res,1))
  ages = np.tile(np.random.randint(18, 61, size=(1,1)), (num_res,1))
  per_ts = np.tile((100-15) * np.random.random_sample((1,1)) + 15, (num_res,1))
  num_cat = np.tile((5-1) * np.random.random_sample((1,20)) + 1, (num_res,1))
  t = np.concatenate((res_ids, ages, per_ts, num_cat), axis=1)
  users_data = np.concatenate((users_data, t), axis=0)

user_df = pd.DataFrame(data=users_data, columns = ['res_id', 'age', 'avg_per_transaction'] + list(cats_20))
user_df.head()

,res_id,age,avg_per_transaction,American,Burgers,Fast Food,Sandwich,Sandwiches,Healthy,Asian,...,Breakfast and Brunch,Desserts,Pizza,Salads,Chicken,Convenience,Italian,Everyday Essentials,Wings,Family Friendly
0,24896.0,52.0,79.431271,1.092546,2.368937,1.319903,3.149351,4.79964,3.114432,4.001346,...,3.946919,4.377756,1.119757,4.824244,3.357502,4.57208,3.488574,4.53365,2.404719,4.275208
1,845.0,52.0,79.431271,1.092546,2.368937,1.319903,3.149351,4.79964,3.114432,4.001346,...,3.946919,4.377756,1.119757,4.824244,3.357502,4.57208,3.488574,4.53365,2.404719,4.275208
2,27374.0,52.0,79.431271,1.092546,2.368937,1.319903,3.149351,4.79964,3.114432,4.001346,...,3.946919,4.377756,1.119757,4.824244,3.357502,4.57208,3.488574,4.53365,2.404719,4.275208
3,21487.0,52.0,79.431271,1.092546,2.368937,1.319903,3.149351,4.79964,3.114432,4.001346,...,3.946919,4.377756,1.119757,4.824244,3.357502,4.57208,3.488574,4.53365,2.404719,4.275208
4,35158.0,52.0,79.431271,1.092546,2.368937,1.319903,3.149351,4.79964,3.114432,4.001346,...,3.946919,4.377756,1.119757,4.824244,3.357502,4.57208,3.488574,4.53365,2.404719,4.275208


In [ ]:
print(f"user transactions shape: {user_df.shape}")

user transactions shape: (47120, 23)


## Create item transactions of users dataframe

In [ ]:
item_df = pd.merge(df, user_df[['res_id']], left_index=True, right_on='res_id', how='right')
item_df = item_df.drop(columns=['res_id'])
item_df.head()

,id,name,score,ratings,category,lat,lng,American,Burgers,Fast Food,...,Desserts,Pizza,Salads,Chicken,Convenience,Italian,Everyday Essentials,Wings,Family Friendly,avg_price
0,26673,Big Daddy Sandwiches,4.560996,74.870989,"Mexican, Latin American, Tacos",39.075291,-77.118092,1,0,0,...,0,0,0,0,0,0,0,0,0,6.991379
1,888,Pizza Hut (200 Highway 31 SW),4.560996,74.870989,"Italian, pizza, wings",34.441800,-86.941700,0,0,0,...,0,0,0,0,0,1,0,0,0,15.301277
2,29319,CNX Thai Cooking,4.800000,131.000000,"Thai, Asian, Noodles, Allergy Friendly",44.480620,-73.211070,0,0,0,...,0,0,0,0,0,0,0,0,0,8.800000
3,23086,Dolce &amp; Ciabatta,4.600000,18.000000,"Black-owned, Sandwich, Coffee and Tea, Europea...",39.107860,-77.569180,0,0,0,...,0,0,0,0,0,0,0,0,0,10.609077
4,37651,Potbelly Sandwich Shop (Sundance Square | 97),4.900000,78.000000,"Deli, Salads, Sandwich, Healthy",32.753409,-97.332815,0,0,0,...,0,0,1,0,0,0,0,0,0,5.604493


In [ ]:
print(f"item transactions shape: {item_df.shape}")

item transactions shape: (47120, 28)


## Creating y labels

In [ ]:
y_label = (5-1) * np.random.random_sample((item_df.shape[0],1)) + 1
y_label[:5]

array([[4.38932891],
       [2.63907133],
       [3.76926708],
       [1.09424693],
       [2.04157719]])

## Training The Model

### Splitting data into training and test

In [ ]:
num_user_f = user_df.shape[1]-3
num_item_f = item_df.shape[1]-6

user_t_f = ['res_id','American', 'Burgers',
       'Fast Food', 'Sandwich', 'Sandwiches', 'Healthy', 'Asian',
       'Comfort Food', 'Family Meals', 'Mexican', 'Breakfast and Brunch',
       'Desserts', 'Pizza', 'Salads', 'Chicken', 'Convenience', 'Italian',
       'Everyday Essentials', 'Wings', 'Family Friendly']
item_t_f = ['id','score', 'ratings', 'American',
       'Burgers', 'Fast Food', 'Sandwich', 'Sandwiches', 'Healthy', 'Asian',
       'Comfort Food', 'Family Meals', 'Mexican', 'Breakfast and Brunch',
       'Desserts', 'Pizza', 'Salads', 'Chicken', 'Convenience', 'Italian',
       'Everyday Essentials', 'Wings', 'Family Friendly']

print(f"user features: {num_user_f} | item features: {num_item_f}")

user features: 20 | item features: 22


In [ ]:
item_weights = np.array([0, 0.1, 0.1] + [2]*20)
user_weights = np.array([0] + [2]*20)

In [ ]:
# scaling the data
scalerItem = StandardScaler()
scalerItem.fit(item_df[item_t_f] * item_weights)
item_train = scalerItem.transform(item_df[item_t_f] * item_weights)

scalerUser = StandardScaler()
scalerUser.fit(user_df[user_t_f] * user_weights)
user_train = scalerUser.transform(user_df[user_t_f] * user_weights)

scalerTarget = MinMaxScaler((-1, 1))
scalerTarget.fit(y_label.reshape(-1, 1))
y_train = scalerTarget.transform(y_label.reshape(-1, 1))

In [ ]:
print(f"the shape of\nuser_train: {user_train.shape} | item_train: {item_train.shape} | y_train: {y_train.shape}" )

the shape of
user_train: (47120, 21) | item_train: (47120, 23) | y_train: (47120, 1)


In [ ]:
# splitting
item_train, item_test = train_test_split(item_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test       = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)
print(f"item training data shape: {item_train.shape}")
print(f"item test data shape: {item_test.shape}")

item training data shape: (37696, 23)
item test data shape: (9424, 23)


### Model Architecture

In [ ]:
# MODEL ARCHITECTURE
num_outputs = 32
tf.random.set_seed(1)
user_NN = tf.keras.models.Sequential([
       
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(num_outputs, activation='linear')
    
])

item_NN = tf.keras.models.Sequential([
       
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(num_outputs, activation='linear')
    
])

# create the user input and point to the base network
input_user = tf.keras.layers.Input(shape=(num_user_f))
vu = user_NN(input_user)
vu = tf.linalg.l2_normalize(vu, axis=1)

# create the item input and point to the base network
input_item = tf.keras.layers.Input(shape=(num_item_f))
vm = item_NN(input_item)
vm = tf.linalg.l2_normalize(vm, axis=1)

# compute the dot product of the two vectors vu and vm
output = tf.keras.layers.Dot(axes=1)([vu, vm])

# specify the inputs and output of the model
model = tf.keras.Model([input_user, input_item], output)

model.summary()

Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_25 (InputLayer)          [(None, 20)]         0           []                               
                                                                                                  
 input_26 (InputLayer)          [(None, 22)]         0           []                               
                                                                                                  
 sequential_24 (Sequential)     (None, 32)           13024       ['input_25[0][0]']               
                                                                                                  
 sequential_25 (Sequential)     (None, 32)           13280       ['input_26[0][0]']               
                                                                                           

In [ ]:
# COMPILING
tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt,
              loss=cost_fn)

## TRAINING and TESTING...

In [ ]:
# FITTING
tf.random.set_seed(1)
model.fit([user_train[:,1:], item_train[:,1:]], y_train, epochs=30)

Epoch 1/30
1178/1178 [==============================] - 2s 1ms/step - loss: 0.3383
Epoch 2/30
1178/1178 [==============================] - 2s 1ms/step - loss: 0.3360
Epoch 3/30
1178/1178 [==============================] - 2s 2ms/step - loss: 0.3358
Epoch 4/30
1178/1178 [==============================] - 2s 2ms/step - loss: 0.3356
Epoch 5/30
1178/1178 [==============================] - 2s 1ms/step - loss: 0.3354
Epoch 6/30
1178/1178 [==============================] - 2s 1ms/step - loss: 0.3353
Epoch 7/30
1178/1178 [==============================] - 2s 1ms/step - loss: 0.3351
Epoch 8/30
1178/1178 [==============================] - 2s 1ms/step - loss: 0.3349
Epoch 9/30
1178/1178 [==============================] - 2s 1ms/step - loss: 0.3347
Epoch 10/30
1178/1178 [==============================] - 2s 1ms/step - loss: 0.3343
Epoch 11/30
1178/1178 [==============================] - 2s 2ms/step - loss: 0.3340
Epoch 12/30
1178/1178 [==============================] - 2s 1ms/step - loss: 0.3336
E

In [ ]:
# EVALUATING
model.evaluate([user_test[:,1:], item_test[:,1:]], y_test)

295/295 [==============================] - 0s 917us/step - loss: 0.3499


0.3499331474304199

## PREDICTING

In [ ]:
user_df.columns

Index(['res_id', 'age', 'avg_per_transaction', 'American', 'Burgers',
       'Fast Food', 'Sandwich', 'Sandwiches', 'Healthy', 'Asian',
       'Comfort Food', 'Family Meals', 'Mexican', 'Breakfast and Brunch',
       'Desserts', 'Pizza', 'Salads', 'Chicken', 'Convenience', 'Italian',
       'Everyday Essentials', 'Wings', 'Family Friendly'],
      dtype='object')

In [ ]:
id = [301]
age = []
tran = [] # usd
f_tran = [0]*20
f_tran[1] = 5.0
# f_tran[2] = 5.0

user_vec = np.array(id + age + tran + f_tran)
user_vec = np.tile(user_vec, (df.shape[0],1))
item_vec = df[item_t_f]
# scale our user and item vectors
suser_vecs = scalerUser.transform(user_vec)
sitem_vecs = scalerItem.transform(item_vec)

# make a prediction
y_p = model.predict([suser_vecs[:,1:], sitem_vecs[:,1:]])

# unscale y prediction 
y_pu = scalerTarget.inverse_transform(y_p)

# sort the results, highest prediction first
sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  #negate to get largest rating first
sorted_ypu   = y_pu[sorted_index]

sorted_index = [str(x) for x in sorted_index[:5]]
sorted_ypu = [str(x) for x in sorted_ypu[:5]]

print(f"user vector: {user_vec[0]}")
print(f"y value: {sorted_ypu}")
print(f"index: {sorted_index}")

df.iloc[sorted_index]

  60/1174 [>.............................] - ETA: 0s  

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1174/1174 [==============================] - 1s 916us/step
user vector: [301.   0.   5.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.]
y value: ['[2.948278]', '[2.947183]', '[2.9457746]', '[2.9457276]', '[2.9454162]']
index: ['19006', '30649', '24823', '2186', '19845']


,id,name,score,ratings,category,lat,lng,American,Burgers,Fast Food,...,Desserts,Pizza,Salads,Chicken,Convenience,Italian,Everyday Essentials,Wings,Family Friendly,avg_price
19006,20401,"We, The Pizza",4.4,225.0,"Italian, Wings, Salads, Pizza, Healthy",38.855060,-77.049878,0,0,0,...,0,1,1,0,0,1,0,1,0,13.736667
30649,32758,Subway (Barbosa 65),4.4,277.0,"Sandwich, Salads, Breakfast and Brunch, Desser...",18.395449,-66.041189,1,0,1,...,1,1,1,0,0,1,0,0,0,6.804630
24823,26587,Pizza Boli's Wisconsin Avenue,4.7,335.0,"Pizza, American, Italian, Salads, Wings, Healt...",38.953568,-77.081746,1,0,0,...,0,1,1,0,0,1,0,1,1,16.194013
2186,2288,Pizza Shuttle,4.3,200.0,"Pizza, Salads, Wings, Fast Food, Desserts, Fam...",43.054655,-87.891048,0,0,1,...,1,1,1,0,0,0,0,1,0,18.257616
19845,21314,Chili's (4085 Virginia Beach Blvd.),4.1,200.0,"American, Burgers, Salads, Family Meals, Healt...",36.841580,-76.115360,1,1,0,...,0,0,1,0,0,0,0,1,0,11.444406
